In [1]:
import gurobipy as gp
from gurobipy import GRB
import random
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, LineString
import numpy as np
import warnings
from tqdm import tqdm_notebook as tqdm
import requests
import json
import API
import matplotlib.pyplot as plt
import GoogleDistanceMatrix
import GoogleVRP
import routing
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)
warnings.filterwarnings('ignore')

## modelling

In [2]:
order = pd.read_csv('Data/ProcessedNov30.csv')

In [3]:
order['門店名稱'].unique()

array(['PH64', 'PH57', 'PHD629', 'PHD606'], dtype=object)

In [4]:
phdstore = 'PHD629'
phstore = 'PH57'

In [5]:
order = order.loc[order['門店名稱'].isin([phdstore,phstore])]
order.shape

(2126, 31)

In [6]:
order['foodPreparationTime'] = pd.to_datetime(order['取餐出發時間']) - pd.to_datetime(order['下單時間'])

In [7]:
order = order[['orderTime', 'arrivalTime', 'foodPreparationTime','waitingTime', 'departureTime', 'tripDurationReal',
       'DeliveryLatLon', 'storeCode', 'StoreLatLon',
       'tripDurationFromGoogle', 'shared','Rider']]

In [8]:
slot = 5
order['departureTime'] = pd.to_datetime(order['departureTime'])
order['hour'] = order['departureTime'].dt.hour
order['min'] = order['departureTime'].dt.minute
order['min'] = order['min'].apply(lambda x: (int(x/slot)+1)*slot)

order.loc[order['min']==60,'hour'] = order.loc[order['min']==60,'hour']+1
order.loc[order['min']==60,'min'] = 0

In [166]:
order['timeWindow'] = pd.Timedelta('0 days 00:55:00') - order['foodPreparationTime']
order['timeWindow'] = order['timeWindow'].dt.seconds


In [167]:
order['timeParameter'] = order['tripDurationReal']/order['tripDurationFromGoogle']
order['timeParameter'] = order['timeParameter'].median()

In [168]:
order[order['tripDurationReal'] < 60]['tripDurationReal'] = order['timeParameter'].median() * order[order['tripDurationReal'] < 60]['tripDurationFromGoogle']

In [169]:
order['date'] = pd.to_datetime(order['orderTime']).dt.date

In [116]:
# dispatch immediately
order['timeWindow'] = order['timeWindow'] /order['timeParameter']
order['timeWindow'] = order['tripDurationFromGoogle']+10

In [170]:
# slot aggregation
order['timeWindow'] = order['timeWindow'] /order['timeParameter']
order.loc[order['timeWindow'] < order['tripDurationFromGoogle'],'timeWindow'] = \
order.loc[order['timeWindow'] < order['tripDurationFromGoogle'],'tripDurationFromGoogle']+10

In [171]:
rider_df = pd.read_excel('../1. Sales Data/2-30Nov - Clock in record.xls')
# rider_df
rider_df['Clock in.1'] = pd.to_datetime(rider_df['Clock in.1'])
rider_df['Date'] = rider_df['Clock in.1'].dt.day

In [172]:
PHDLatLon = order.loc[order['storeCode']==phdstore]['StoreLatLon'].values[0]
PHLatLon = order.loc[order['storeCode']==phstore]['StoreLatLon'].values[0]


In [173]:
order['tripTime'] = 0
order['rider'] = 0

In [174]:
PHD_output = pd.DataFrame()
PH_output = pd.DataFrame()

In [175]:
order['tripTime'] = 0
order['rider'] = 0
order['delayed'] = 0
order['departureTimeSimulation'] = pd.to_datetime(order.orderTime).dt.date.astype(str)+' '+order['hour'].apply(lambda x:str(x).zfill(2))+\
                                        ':'+order['min'].apply(lambda x:str(x).zfill(2))+':00'
PHD_output = pd.DataFrame()
PH_output = pd.DataFrame()

In [176]:
order.shape

(2126, 21)

In [177]:
month = '11'
for day in range(2,31):
    day = str(day).zfill(2)
    # reset riders
    riders = order.loc[order['date']==pd.to_datetime('2020-%s-%s'%(month,day))]['Rider'].unique().tolist()
    riders = [rider for rider in riders if rider!='3rd']
    riderStatus = {}
    for i in (riders):
        riderStatus[i] = 0
    for hour in range(11,24):
        for minute in np.arange(0,59,slot).tolist()+[59]:
#             print(day,hour,minute)
            # update rider status at current time
            for key in riderStatus.keys():
                if riderStatus[key]!= 0:
                    if riderStatus[key] <= pd.to_datetime('2020-%s-%s %s:%s:00'%(month,day,hour,str(minute).zfill(2))):
                        riderStatus[key] = 0

            candidate = order.loc[(pd.to_datetime(order['orderTime']).dt.date.astype(str)=='2020-%s-%s'%(month,day))&(order['hour']==hour)&(order['min']==minute)]
            if len(candidate) > 0:
                PHD = candidate.loc[candidate['storeCode']==phdstore]
                PH = candidate.loc[candidate['storeCode']==phstore]
                
                timeParameter = order.loc[(order['hour']==hour)&(order['min']==minute)]['timeParameter'].unique()[0]
                print('PHD',len(PHD))
                PHD_output,riderStatus,order = routing.tripAssign(month,day,hour,minute,timeParameter,PHD, PHDLatLon, riderStatus,order,PHD_output)
                
                print('PH',len(PH))
                PH_output,riderStatus,order = routing.tripAssign(month,day,hour,minute,timeParameter,PH, PHLatLon, riderStatus,order,PH_output)

                

PHD 0
PH 1
PHD 0
PH 1
PHD 0
PH 1
PHD 0
PH 1
PHD 0
PH 2
PHD 1
PH 0
PHD 1
PH 0
PHD 1
PH 2
PHD 1
PH 0
PHD 0
PH 1
PHD 0
PH 1
PHD 1
PH 0
PHD 1
PH 0
PHD 0
PH 1
PHD 0
PH 1
PHD 1
PH 0
PHD 1
PH 0
PHD 1
PH 1
PHD 2
PH 0
PHD 0
PH 1
PHD 0
PH 1
PHD 0
PH 2
PHD 1
PH 2
PHD 0
PH 3
PHD 1
PH 2
PHD 0
PH 1
PHD 2
PH 0
PHD 1
PH 1
PHD 0
PH 2
PHD 2
PH 1
PHD 0
PH 1
PHD 1
PH 0
PHD 0
PH 1
PHD 0
PH 3
PHD 0
PH 1
PHD 0
PH 1
PHD 1
PH 0
PHD 1
PH 1
PHD 1
PH 1
PHD 0
PH 1
PHD 0
PH 1
PHD 0
PH 1
PHD 2
PH 0
PHD 1
PH 0
PHD 0
PH 1
PHD 2
PH 0
PHD 0
PH 1
PHD 0
PH 1
PHD 1
PH 0
PHD 0
PH 1
PHD 0
PH 2
PHD 1
PH 0
PHD 0
PH 1
PHD 0
PH 1
PHD 0
PH 1
PHD 0
PH 2
PHD 0
PH 1
PHD 2
PH 0
PHD 0
PH 1
PHD 2
PH 1
PHD 0
PH 1
PHD 0
PH 2
PHD 1
PH 0
PHD 0
PH 2
PHD 2
PH 0
PHD 0
PH 1
PHD 1
PH 0
PHD 0
PH 2
PHD 1
PH 1
PHD 0
PH 1
PHD 1
PH 0
PHD 0
PH 1
PHD 0
PH 1
PHD 0
PH 1
PHD 0
PH 1
PHD 0
PH 1
PHD 0
PH 2
PHD 0
PH 1
PHD 0
PH 1
PHD 1
PH 0
PHD 0
PH 1
PHD 1
PH 1
PHD 0
PH 1
PHD 1
PH 0
PHD 0
PH 1
PHD 0
PH 1
PHD 1
PH 0
PHD 0
PH 1
PHD 0
PH 2
PHD 0
PH 1
PHD 2
PH 3

PHD 0
PH 2
PHD 0
PH 1
PHD 3
PH 2
PHD 0
PH 2
PHD 3
PH 0
PHD 2
PH 1
PHD 1
PH 1
PHD 1
PH 3
PHD 1
PH 1
PHD 2
PH 2
PHD 2
PH 0
PHD 1
PH 2
PHD 1
PH 3
PHD 2
PH 0
PHD 3
PH 1
PHD 3
PH 1
PHD 2
PH 0
PHD 1
PH 2
PHD 3
PH 1
PHD 0
PH 1
PHD 1
PH 1
PHD 0
PH 1
PHD 2
PH 3
PHD 0
PH 3
PHD 5
PH 2
PHD 0
PH 4
PHD 0
PH 1
PHD 0
PH 3
PHD 1
PH 0
PHD 0
PH 2
PHD 1
PH 0
PHD 1
PH 0
PHD 1
PH 0
PHD 3
PH 0
PHD 0
PH 1
PHD 1
PH 0
PHD 1
PH 0
PHD 1
PH 2
PHD 0
PH 2
PHD 1
PH 0
PHD 0
PH 1
PHD 0
PH 1
PHD 1
PH 0
PHD 2
PH 0
PHD 2
PH 1
PHD 1
PH 0
PHD 0
PH 1
PHD 0
PH 1
PHD 1
PH 0
PHD 0
PH 2
PHD 1
PH 0
PHD 0
PH 3
PHD 1
PH 0
PHD 2
PH 0
PHD 0
PH 1
PHD 1
PH 0
PHD 1
PH 0
PHD 1
PH 0
PHD 2
PH 0
PHD 1
PH 0
PHD 3
PH 0
PHD 0
PH 2
PHD 0
PH 1
PHD 3
PH 2
PHD 0
PH 5
PHD 0
PH 1
PHD 3
PH 0
PHD 2
PH 1
PHD 3
PH 0
PHD 1
PH 0
PHD 2
PH 0
PHD 2
PH 0
PHD 2
PH 0
PHD 6
PH 6
PHD 1
PH 0
PHD 0
PH 5
PHD 0
PH 1
PHD 4
PH 0
PHD 5
PH 2
PHD 1
PH 2
PHD 0
PH 5
PHD 0
PH 2
PHD 0
PH 6
PHD 0
PH 1
PHD 0
PH 3
PHD 0
PH 2
PHD 0
PH 6
PHD 0
PH 3
PHD 0
PH 2
PHD 1
PH 0
PHD 1
PH 0

In [179]:
output = pd.concat([PHD_output,PH_output])

In [180]:
output.shape

(2126, 21)

In [181]:
output['arrivalTimeSimulation'] = pd.to_datetime(output['departureTimeSimulation']) +\
                                        pd.to_timedelta(output['tripTime'],'seconds')
output['waitingTimeSimulation'] = output['arrivalTimeSimulation'] - pd.to_datetime(output['orderTime'])
output['waitingTimeReal'] = pd.to_datetime(output['arrivalTime']) - pd.to_datetime(output['orderTime'])


In [182]:
output['waitingTimeSimulation'].mean()

Timedelta('0 days 00:44:41.106487')

In [183]:
output['waitingTimeReal'].mean()

Timedelta('0 days 00:45:06.647224')

In [184]:
output.to_csv('Output/order57_629_nov30_5min_55TW.csv',index=False)